In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

In [2]:
import importlib
from omegaconf import OmegaConf
import torch
import numpy as np
from hp.core.evaluator import PriorEvaluator, IKEvaluator
from hp.core.cond_trainer import CondSO3PriorTrainer
from hp.analysis.eval_ik_tools import compute_geod_and_mpjpe_for_based_on_k_samples

Visualizer not available. Install body_visualizer to use it.


In [3]:
path_pretrained_models = "experiments/pretrained_models"

Computed results might slightly deviate from the paper since we moved from `pytorch3d` to `roma` for rotation conversions, which results in a different set of randomly generated rotations.

# Prior

In [4]:
# Get the model
exp_id = 900
experiment_path = f"{path_pretrained_models}/id_{exp_id:04}"

config = OmegaConf.load(f"{experiment_path}/config.yaml")

config.device = "cuda:0"

config.only_eval = True
config.standard_mode = True
config.condition = False

module = importlib.import_module("hp.core.trainer")
class_ = getattr(module, config.trainer)
trainer = class_(config)
trainer.load_model(save_path=experiment_path)
    

Only evaluation mode.


In [5]:
# Sample from the prior
with torch.no_grad():
    samples = trainer.sample(100)

In [6]:
# Evaluate the model
d = PriorEvaluator.eval(config, trainer, render_human=False, plot_samples=False, save_plot=False, compute_d_wasser=False)
print(d)


Average log-likelihood: 137.91
{'eval_avg_ll': 137.91448465983072, 'eval_avg_ll_0': 137.9574737548828, 'avg_ll_1': 137.82850646972656, 'eval_avg_ll_std': 0.05361682168411739}


# IK

In [ ]:
# Get the model

exp_id = 410
experiment_path = f"{path_pretrained_models}/id_{exp_id:04}"

config = OmegaConf.load(f"{experiment_path}/config.yaml")

config.device = "cuda:0"
config.batch_size = 2000
config.num_eval_samples = 10_000

np.random.seed(config.seed)
torch.manual_seed(config.seed)

config.standard_mode = True
config.only_eval = True
config.selected_joints = "None"
config.conditioning.vary_mask_prob = False

trainer = CondSO3PriorTrainer(config)
trainer.load_model()

seed = 10
np.random.seed(seed); torch.manual_seed(seed);


Creating CondSO3PriorTrainer...
Only evaluation mode.
Using FlowND as pdf estimator with 2972.656k parameters.
Conditioning mode: 3D
Created CondSO3PriorTrainer.


In [8]:
# Evaluate the model
print("Avg LL: ", trainer.eval_estimator(num_samples=-1, complete=True))
d = IKEvaluator.eval_with_dataset(trainer, n_samples=5_000, dataset = "eval")

Avg LL:  217.4688262939453
==Evaluation with eval dataset==
-> Eval with 5000 samples
 Sample Avg LL: 234.589
 Forward Avg LL: 217.919
 MPJPE: tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  9.0000,  8.6000,  1.1000, 11.9000,
         9.0000,  3.0000, 25.1000, 13.9000,  4.6000,  3.7000,  3.9000, 14.2000,
         8.6000,  7.8000, 11.3000, 11.9000, 16.6000, 15.3000], device='cuda:0')
 Geodesic distance: tensor([0.0605, 0.0531, 0.0216, 0.0759, 0.0784, 0.0635, 0.2083, 0.1400, 0.0684,
        0.1949, 0.0989, 0.1071, 0.2119, 0.1321, 0.1414, 0.2342, 0.2099, 0.4896,
        0.5001])
 Mean MPJPE: 8.16mm
 Mean MPJPE 2D: 6.87mm
 Mean geodesic distance: 0.1626 rad = 9.3 deg


In [9]:
dataset = "eval"; n_samples = 5_000
dataset = trainer.eval_dataset if dataset == "eval" else trainer.train_dataset
idx = list(np.random.permutation(len(dataset))[:n_samples])
dataset_samples = dataset[idx]
theta_raw, p3d = dataset_samples["theta"].to(trainer.config.device).reshape(n_samples,-1,3,3), dataset_samples["condition"].to(trainer.config.device)

In [10]:
# Evaluate the model with various samples
trainer.config.conditioning.mask = False
n_samples = theta_raw.shape[0]
for k_samples in [1, 10]:
    ths, p3d_raw, gs, mjps = compute_geod_and_mpjpe_for_based_on_k_samples(n_samples, k_samples, p3d, theta_raw, 
                                                                           trainer=trainer, selected_joints=trainer.selected_joints)

-- 1 samples --
LL=221.38035583496094
Mean MPJPE over samples=8.183 | Mean GEO=0.163 | Min MPJPE=8.183 | Mean MPJPE=8.183 | Min GEO=0.163 | Mean GEO=0.163 | P-MPJPE=8.632 | Mean P-MPJPE=8.632
-- 10 samples --
LL=261.8565368652344
Mean MPJPE over samples=8.165 | Mean GEO=0.163 | Min MPJPE=2.669 | Mean MPJPE=5.470 | Min GEO=0.065 | Mean GEO=0.121 | P-MPJPE=8.443 | Mean P-MPJPE=5.621


# IK with occlusion

In [ ]:
exp_id = 384
experiment_path = f"{path_pretrained_models}/id_{exp_id:04}"

config = OmegaConf.load(f"{experiment_path}/config.yaml")

config.device = "cuda:0"
config.batch_size = 2000
config.num_eval_samples = 5_000

np.random.seed(config.seed)
torch.manual_seed(config.seed)

config.standard_mode = True 
config.only_eval = True
config.selected_joints = "None"
config.conditioning.vary_mask_prob = False

trainer = CondSO3PriorTrainer(config)
trainer.load_model()

seed = 10
np.random.seed(seed); torch.manual_seed(seed);


Creating CondSO3PriorTrainer...
Only evaluation mode.
Using FlowND as pdf estimator with 1517.104k parameters.
Conditioning mode: 3D
Created CondSO3PriorTrainer.


In [12]:
trainer.config.conditioning.zero_mask_prob = 0.3
trainer.config.conditioning.vary_mask_prob = True
trainer.config.conditioning.mask_type = "random"
d = IKEvaluator.eval_with_dataset(trainer, n_samples=5000, dataset = "eval")
print(d)

==Evaluation with eval dataset==
-> Eval with 5000 samples
 Sample Avg LL: 184.215
 Forward Avg LL: 168.765
 MPJPE: tensor([  0.0000,   0.0000,   0.0000,   0.0000,  54.9000,  54.4000,   8.7000,
         96.6000, 102.1000,  11.5000, 106.3000, 107.8000,  24.8000,  20.6000,
         19.2000,  43.4000,  34.7000,  35.2000,  93.4000, 102.5000, 176.4000,
        180.6000], device='cuda:0')
 Geodesic distance: tensor([0.1967, 0.1835, 0.0831, 0.3283, 0.3006, 0.1305, 0.2819, 0.2982, 0.0986,
        0.2963, 0.2041, 0.2246, 0.3002, 0.3280, 0.3504, 0.5020, 0.5232, 0.5597,
        0.5951])
 Mean MPJPE: 57.86mm
 Mean MPJPE 2D: 40.19mm
 Mean geodesic distance: 0.3045 rad = 17.4 deg
{'mpjpe': 0.057862289249897, 'geod_dist': 0.30447348952293396, 'mpjpe_2d': 0.04018593952059746, 'p_mpjpe': 0.060957398265600204, 'll': 168.7648162841797}


In [13]:
dataset = "eval"; n_samples = 5_000
dataset = trainer.eval_dataset if dataset == "eval" else trainer.train_dataset
idx = list(np.random.permutation(len(dataset))[:n_samples])
dataset_samples = dataset[idx]
theta_raw, p3d = dataset_samples["theta"].to(trainer.config.device).reshape(n_samples,-1,3,3), dataset_samples["condition"].to(trainer.config.device)

In [14]:
# Randomly occluded
trainer.config.conditioning.zero_mask_prob = 0.3
trainer.config.conditioning.vary_mask_prob = True
trainer.config.conditioning.mask_type = "random"
n_samples = theta_raw.shape[0]
for k_samples in [1, 10]:
    ths, p3d_raw, gs, mjps = compute_geod_and_mpjpe_for_based_on_k_samples(n_samples, k_samples, p3d, theta_raw, 
                                                                           trainer=trainer, selected_joints=trainer.selected_joints)

-- 1 samples --
LL=80.73220825195312
Mean MPJPE over samples=57.009 | Mean GEO=0.303 | Min MPJPE=57.009 | Mean MPJPE=57.009 | Min GEO=0.303 | Mean GEO=0.303 | P-MPJPE=60.187 | Mean P-MPJPE=60.187
-- 10 samples --
LL=87.29480743408203
Mean MPJPE over samples=57.755 | Mean GEO=0.305 | Min MPJPE=7.610 | Mean MPJPE=39.500 | Min GEO=0.101 | Mean GEO=0.200 | P-MPJPE=59.900 | Mean P-MPJPE=41.359


In [15]:
# Occluded joints
from hp.analysis.eval_ik_tools import compute_geod_and_mpjpe_for_based_on_k_samples

n_samples = 10_000
trainer.config.conditioning.mask = True
trainer.config.conditioning.vary_mask_prob = False
trainer.config.conditioning.mask_type = "selected_joints"

sel_joints_nomask = [""]
sel_joints_1 = ["LeftUpLeg", "LeftLeg", "LeftFoot"] #(1)
sel_joints_2 = ["LeftHand", "LeftArm", "LeftElbow"] #(2)
sel_joints_3 = ["RightShoulder", "RightArm"] # (3)
sel_joints_4 = ["LeftHand", "LeftArm", "LeftElbow", "RightToeBase", "RightFoot"]
sel_joints_smplik = ["LeftHand", "RightHand", "LeftFoot", "RightFoot", "Head"]

b_compute_mesh_error = False


for i_sel, sel_joints in enumerate([sel_joints_nomask, sel_joints_1, sel_joints_2, sel_joints_3, sel_joints_smplik]):
    
    print(f"=====================================")
    print(f"Selection {i_sel}: Eval with {n_samples} samples for mask_joints: {sel_joints}")
    
    for i_eval in range(1):
        print(f"== Evaluation {i_eval} ==")
        idx = list(np.random.permutation(len(dataset))[:n_samples])
        s = dataset[idx]
        theta_raw, p3d = s["theta"].to(trainer.config.device).reshape(n_samples,-1,3,3), s["condition"].to(trainer.config.device)
        trainer.config.conditioning.mask_joints = sel_joints
        n_samples = theta_raw.shape[0]
        for k_samples in [1,10]:
            ths, p3d_raw, gs, mjps, g_mean, mjpe_mean, p_mjpe_mean = compute_geod_and_mpjpe_for_based_on_k_samples(n_samples, k_samples, p3d, theta_raw, trainer=trainer, return_eval_values=True)

Selection 0: Eval with 10000 samples for mask_joints: ['']
== Evaluation 0 ==
-- 1 samples --
LL=197.48883056640625
Mean MPJPE over samples=17.852 | Mean GEO=0.228 | Min MPJPE=17.852 | Mean MPJPE=17.852 | Min GEO=0.228 | Mean GEO=0.228 | P-MPJPE=18.617 | Mean P-MPJPE=18.617
-- 10 samples --
LL=191.54006958007812
Mean MPJPE over samples=17.884 | Mean GEO=0.228 | Min MPJPE=6.284 | Mean MPJPE=11.913 | Min GEO=0.091 | Mean GEO=0.158 | P-MPJPE=18.551 | Mean P-MPJPE=12.294
Selection 1: Eval with 10000 samples for mask_joints: ['LeftUpLeg', 'LeftLeg', 'LeftFoot']
== Evaluation 0 ==
-- 1 samples --
LL=194.8851318359375
Mean MPJPE over samples=20.710 | Mean GEO=0.239 | Min MPJPE=20.710 | Mean MPJPE=20.710 | Min GEO=0.239 | Mean GEO=0.239 | P-MPJPE=21.667 | Mean P-MPJPE=21.667
-- 10 samples --
LL=213.22666931152344
Mean MPJPE over samples=20.835 | Mean GEO=0.239 | Min MPJPE=7.382 | Mean MPJPE=13.920 | Min GEO=0.097 | Mean GEO=0.166 | P-MPJPE=21.638 | Mean P-MPJPE=14.491
Selection 2: Eval with 10

# 2D to 3D uplifting

In [ ]:
exp_id = 510
experiment_path = f"{path_pretrained_models}/id_{exp_id:04}"


config = OmegaConf.load(f"{experiment_path}/config.yaml")

config.device = "cuda:0"
config.batch_size = 2000
config.num_eval_samples = 10_000


np.random.seed(config.seed)
torch.manual_seed(config.seed)


config.standard_mode = True
config.only_eval = True
config.selected_joints = "None"
config.conditioning.vary_mask_prob = False

trainer = CondSO3PriorTrainer(config)
trainer.load_model()

seed = 10
np.random.seed(seed); torch.manual_seed(seed);


Creating CondSO3PriorTrainer...
Only evaluation mode.
Using FlowND as pdf estimator with 2971.248k parameters.
Conditioning mode: 2D
Created CondSO3PriorTrainer.


In [17]:
# 1 sample
print("Avg LL: ", trainer.eval_estimator(num_samples=-1, complete=True))
d = IKEvaluator.eval_with_dataset(trainer, n_samples=5_000, dataset = "eval")
print(d)

Avg LL:  202.2058868408203
==Evaluation with eval dataset==
-> Eval with 5000 samples
 Sample Avg LL: 218.654
 Forward Avg LL: 202.595
 MPJPE: tensor([ 0.0000,  0.0000,  0.0000,  0.0000, 49.1000, 43.1000,  9.6000, 62.7000,
        54.7000,  8.6000, 65.1000, 57.5000, 15.8000, 13.2000, 12.9000, 15.5000,
        22.7000, 23.8000, 48.3000, 47.8000, 85.7000, 91.1000], device='cuda:0')
 Geodesic distance: tensor([0.1475, 0.1294, 0.0831, 0.1723, 0.1599, 0.1410, 0.1733, 0.1524, 0.0884,
        0.1625, 0.1555, 0.1787, 0.2219, 0.1796, 0.1945, 0.3823, 0.3445, 0.5039,
        0.5066])
 Mean MPJPE: 33.05mm
 Mean MPJPE 2D: 6.79mm
 Mean geodesic distance: 0.2146 rad = 12.3 deg
{'mpjpe': 0.033050287514925, 'geod_dist': 0.21459785103797913, 'mpjpe_2d': 0.006794109009206295, 'p_mpjpe': 0.03089766576886177, 'll': 202.59474182128906}


In [18]:
dataset = "eval"; n_samples = 5_000
dataset = trainer.eval_dataset if dataset == "eval" else trainer.train_dataset
idx = list(np.random.permutation(len(dataset))[:n_samples])
dataset_samples = dataset[idx]
theta_raw, p3d = dataset_samples["theta"].to(trainer.config.device).reshape(n_samples,-1,3,3), dataset_samples["condition"].to(trainer.config.device)

In [19]:
trainer.config.conditioning.mask = False
n_samples = theta_raw.shape[0]
for k_samples in [1, 10]:
    ths, p3d_raw, gs, mjps = compute_geod_and_mpjpe_for_based_on_k_samples(n_samples, k_samples, p3d, theta_raw, 
                                                                           trainer=trainer, selected_joints=trainer.selected_joints)

-- 1 samples --
LL=213.45082092285156
Mean MPJPE over samples=6.718 | Mean GEO=0.214 | Min MPJPE=6.718 | Mean MPJPE=6.718 | Min GEO=0.214 | Mean GEO=0.214 | P-MPJPE=7.168 | Mean P-MPJPE=7.168
-- 10 samples --
LL=196.19915771484375
Mean MPJPE over samples=6.747 | Mean GEO=0.215 | Min MPJPE=1.687 | Mean MPJPE=5.114 | Min GEO=0.085 | Mean GEO=0.161 | P-MPJPE=7.082 | Mean P-MPJPE=5.055
